# IS 675 Lab 9: Learning from Images with Convolutions

## 1. Recap

Butterfly Classification case

Train, Test data set for 10 butterfly species. All images are 224 X 224 X 3 in jpg format.

In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from itertools import chain
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

In [3]:
# Define the image preprocessing pipeline
Butterfly_trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor() # ToTensor() converts images to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0]
])

In [4]:
# Loading images and pass the images through our preprocessing pipeline
train_butterfly10 = ImageFolder('/content/drive/MyDrive/IS675_data/Week8_data/butterfly_train_top10', transform=Butterfly_trans)
test_butterfly10 = ImageFolder('/content/drive/MyDrive/IS675_data/Week8_data/butterfly_test_top10', transform=Butterfly_trans)

In [ ]:
# Examine the number of classes
print(len(train_butterfly10.classes))

In [ ]:
# Examine the sizes of training and test data
print(len(train_butterfly10), len(test_butterfly10))

In [ ]:
# Display a sample of data from test set
class_names = ['AN 88','BLUE MORPHO','COMMON WOOD-NYMPH','MONARCH','PEACOCK','PIPEVINE SWALLOW','ULYSES','VICEROY','YELLOW SWALLOW TAIL','ZEBRA LONG WING']

fig = plt.figure(figsize=(15,5))
num_classes = 10
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    ax.set_title(class_names[i])
    img = next(img for img, label in test_butterfly10 if label == i)
    if i == 3:
      example_img = img
    plt.imshow(img.permute(1, 2, 0))
plt.show()

In [ ]:
# Examine the tensor of a zebra long wing image
print(img.shape)

## 2. Convolutions

In [9]:
#nn.Conv2d are the number of input features (or channels, since we’re dealing with multichannel images: that is, more than one value per pixel), the number of output features, and the size of the kernel.
conv = nn.Conv2d(3, 16, kernel_size=3)

In [10]:
conv.weight.shape, conv.bias.shape

(torch.Size([16, 3, 3, 3]), torch.Size([16]))

In [11]:
output = conv(example_img)
example_img.shape, output.shape

(torch.Size([3, 224, 224]), torch.Size([16, 222, 222]))

In [12]:
# Maintain the same image size with padding
conv = nn.Conv2d(3, 16, kernel_size=5, padding=2)

In [ ]:
output = conv(example_img)
output.shape

**Q1**. Usng the same input data, what is the output dimension of a convolution with 32 output channel, a 3*3 kernel, and padding = 1?

###2.1. Detecting features with convolutions

In [ ]:
plt.imshow(example_img[0], cmap='gray')
plt.show()

In [15]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1)

with torch.no_grad():
    conv.weight[:] = torch.tensor([[-1.0, 0.0, 1.0],
                                   [-1.0, 0.0, 1.0],
                                   [-1.0, 0.0, 1.0]])
    conv.bias.zero_()

**Q2**. Why we turn off the auto grad here? What happens if we do not do it?

In [ ]:
output = conv(example_img)
plt.imshow(output[0].detach(), cmap='gray')
plt.show()

In [18]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1)

with torch.no_grad():
    conv.weight[:] = torch.tensor([[-1.0, -1.0, -1.0],
                                   [0.0, 0.0, 0.0],
                                   [1.0, 1.0, 1.0]])
    conv.bias.zero_()

In [ ]:
output = conv(example_img)
plt.imshow(output[0].detach(), cmap='gray')
plt.show()

###2.2. Looking Further with Depth and Pooling

In [ ]:
# Downsample our image
pool = nn.MaxPool2d(4)
output = pool(img)
output.shape

**Q3**. Explain why and how the image shape reduced to 56*56?

**Q4**. Here we have not set any stride for pooling and it goes by default setting. What is the default stride for this pooling? What will be the size if we set the stride = 1 for the pooling?

### 2.3. Model training and evaluation

In [ ]:
# Examine the mean and std of images in the training data
imgs = torch.stack([img_t for img_t, label in train_butterfly10], dim=3)
print(imgs.view(3, -1).mean(dim=1), imgs.view(3, -1).std(dim=1))

**Q5**. What other number will work in the code above instead of -1?

In [22]:
# Define the image preprocessing pipeline to include normalization
Butterfly_trans = transforms.Compose([transforms.Resize((224, 224)), # composes several transforms together
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.4621, 0.4528, 0.3400],std=[0.2884, 0.2767, 0.2862])
                                      ])

In [23]:
# Loading images and pass the images through our preprocessing pipeline
train_butterfly10 = ImageFolder('/content/drive/MyDrive/IS675_data/Week8_data/butterfly_train_top10', transform=Butterfly_trans)
test_butterfly10 = ImageFolder('/content/drive/MyDrive/IS675_data/Week8_data/butterfly_test_top10', transform=Butterfly_trans)

In [24]:
# Define training and testing data loader, and set batch size to 128
train_loader_butterfly10 = torch.utils.data.DataLoader(train_butterfly10, batch_size=128, shuffle=True)
test_loader_butterfly10 = torch.utils.data.DataLoader(test_butterfly10, batch_size=128, shuffle=False)

In [25]:
# Build a neural network on training data
class neural_network(nn.Module):
    def __init__(self, hidden_size1, out_size):
        super().__init__()
        self.network = nn.Sequential(
          nn.Conv2d(3,8, kernel_size = 5, padding = 2), # Q6. What is the output size after this convolution?
          nn.ReLU(),
          nn.MaxPool2d(4), # Q7. What is the output size after this pooling?
          nn.Conv2d(8, 4, kernel_size = 3, padding = 1), # Q8. What is the output size after this convolution?
          nn.ReLU(),
          nn.MaxPool2d(4), # Q9. What is the output size after this pooling?
          nn.Flatten(), # This will make it a one dimensional vector
          nn.Linear(X, hidden_size1), # Q10. Calculate X?
          nn.ReLU(),
          nn.Linear(hidden_size1, out_size))

    def forward(self, x):
      out = self.network(x)
      return out

In [26]:
# Define training loop function
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(0, n_epochs):
        # Training Phase
        model.train()
        loss_train = 0.0
        for images, labels in train_loader:

            outputs = model(images)

            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 0 or epoch == n_epochs-1 or epoch % 1 == 0:
            print('Epoch {}, Training loss {}'.format(epoch, loss_train / len(train_loader)))

In [ ]:
# Model training
torch.manual_seed(0)
model = neural_network(32, 10)
optimizer_adam = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

training_loop(n_epochs= 15, optimizer= optimizer_adam, model = model, loss_fn = loss_fn, train_loader= train_loader_butterfly10)

In [28]:
# Define testing function
def test(model, train_loader, test_loader):

  # testing phase
  model.eval()
  predict_train = []
  predict_test = []
  labels_train = []
  labels_test = []

  with torch.no_grad():
      for inputs, labels in train_loader:
          outputs = model(inputs)
          index_, predicted = torch.max(outputs, dim=1)
          predict_train.append(predicted.tolist())
          labels_train.append(labels.tolist())

      for inputs, labels in test_loader:
          outputs = model(inputs)
          index_, predicted = torch.max(outputs, dim=1)
          predict_test.append(predicted.tolist())
          labels_test.append(labels.tolist())

  print("Confusion matrix on train:\n",  confusion_matrix(list(chain(*labels_train)), list(chain(*predict_train)), labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
  print()
  print("Classification report on train:\n",  classification_report(list(chain(*labels_train)), list(chain(*predict_train)), labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
  print()
  print("Confusion matrix on test:\n",  confusion_matrix(list(chain(*labels_test)), list(chain(*predict_test)), labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
  print()
  print("Classification report on test:\n",  classification_report(list(chain(*labels_test)), list(chain(*predict_test)), labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [ ]:
# Examine evaluation results
test(model, train_loader_butterfly10, test_loader_butterfly10)

## 3. Training on GPU
Change the Notebook Settings in Colab: Edit-> Notebook Settings -> GPU

In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from itertools import chain
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

In [3]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cpu.


In [4]:
# Define the image preprocessing pipeline to include normalization
Butterfly_trans = transforms.Compose([transforms.Resize((224, 224)), # composes several transforms together
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.4621, 0.4528, 0.3400],std=[0.2884, 0.2767, 0.2862])
                                      ])

In [5]:
# Loading images and pass the images through our preprocessing pipeline
train_butterfly10 = ImageFolder('/content/drive/MyDrive/IS675_data/Week8_data/butterfly_train_top10', transform=Butterfly_trans)
test_butterfly10 = ImageFolder('/content/drive/MyDrive/IS675_data/Week8_data/butterfly_test_top10', transform=Butterfly_trans)

In [6]:
# Define training and testing data loader, and set batch size to 128
train_loader_butterfly10 = torch.utils.data.DataLoader(train_butterfly10, batch_size=128, shuffle=True)
test_loader_butterfly10 = torch.utils.data.DataLoader(test_butterfly10, batch_size=128, shuffle=False)

In [8]:
# Build a neural network on training data
class neural_network(nn.Module):
    def __init__(self,  hidden_size, out_size):
        super().__init__()
        self.network = nn.Sequential(
          nn.Conv2d(3, 8, kernel_size=5, padding=2),
          nn.ReLU(),
          nn.MaxPool2d(4),
          nn.Conv2d(8, 4, kernel_size=3, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(4),
          nn.Flatten(),
          nn.Linear(X, hidden_size), # (-1, 32)
          nn.ReLU(),
          nn.Linear(hidden_size, out_size))

    def forward(self, x):
        out = self.network(x)
        return out

In [9]:
# Define training loop function
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(0, n_epochs):
        # Training Phase
        model.train()
        loss_train = 0.0
        for inputs, labels in train_loader:

            # These two lines that move inputs and labels to the device we are training on are the only difference from our previous version.
            inputs = inputs.to(device=device)
            labels = labels.to(device=device)

            outputs = model(inputs)

            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 0 or epoch == n_epochs-1 or epoch % 1 == 0:
            print('Epoch {}, Training loss {}'.format(epoch, loss_train / len(train_loader)))

In [ ]:
# Model training
# Model training
torch.manual_seed(0)
model = neural_network(32, 10).to(device=device) # Move our model (all parameters) to the GPU
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

training_loop(n_epochs = 20, optimizer = optimizer, model = model, loss_fn = loss_fn, train_loader = train_loader_butterfly10)

In [ ]:
# Define testing function
def test(model, train_loader, test_loader):

  # testing phase
  model.eval()
  predict_train = []
  predict_test = []
  labels_train = []
  labels_test = []

  with torch.no_grad():
      for inputs, labels in train_loader:
          # These two lines that move inputs and labels to the device we are training on are the only difference from our previous version.
          inputs = inputs.to(device=device)
          labels = labels.to(device=device)

          outputs = model(inputs)
          index_, predicted = torch.max(outputs, dim=1)
          predict_train.append(predicted.tolist())
          labels_train.append(labels.tolist())

      for inputs, labels in test_loader:
          # These two lines that move inputs and labels to the device we are training on are the only difference from our previous version.
          inputs = inputs.to(device=device)
          labels = labels.to(device=device)

          outputs = model(inputs)
          index_, predicted = torch.max(outputs, dim=1)
          predict_test.append(predicted.tolist())
          labels_test.append(labels.tolist())

  print("Confusion matrix on train:\n",  confusion_matrix(list(chain(*labels_train)), list(chain(*predict_train)), labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
  print()
  print("Classification report on train:\n",  classification_report(list(chain(*labels_train)), list(chain(*predict_train)), labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
  print()
  print("Confusion matrix on test:\n",  confusion_matrix(list(chain(*labels_test)), list(chain(*predict_test)), labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
  print()
  print("Classification report on test:\n",  classification_report(list(chain(*labels_test)), list(chain(*predict_test)), labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [ ]:
# Examine evaluation results
test(model, train_loader_butterfly10, test_loader_butterfly10)